In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from src.baselines import CNNModel, CARPModel, MSModel, LinearModel
from src.torch_helpers import NamedTensorDataset
from src.datamodule import PeptideDataModule
from pytorch_lightning import Trainer
from src.torch_helpers import NoValProgressBar
from src.constants import MSConstants
C = MSConstants()

torch.manual_seed(0);

/home/gridsan/mmurphy/.conda/envs/MSPretraining/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from src.torch_helpers import start_tensorboard
# start_tensorboard(login_node='login-2')

In [5]:
# generate negatives by producing tons of shuffled sequences
# then cluster, and only take clusters containing sufficiently many positives

# from src.cdhit import CDHIT

# def generate_negatives(sequences, num_shuffles=10, min_frac=0.1, random_state=0):
#     pos_seqs = list(sequences)
#     neg_seqs = []
#     rng = npr.RandomState(random_state)
#     for n in range(num_shuffles):
#         neg_seqs += [''.join(rng.permutation(list(s))) for s in pos_seqs]

#     seqs = np.array(pos_seqs + neg_seqs)
#     ids = np.array([1]*len(pos_seqs) + [0]*len(neg_seqs))
#     clusters = np.array(CDHIT(threshold=0.5,word_length=3).fit_predict(seqs))
#     pos_frac = pd.DataFrame([clusters,ids],index=['clusters','ids']).T.groupby('clusters').mean()['ids']
#     pos_clusters = set(pos_frac[pos_frac>min_frac].index)
    
#     pos_seqs = set(pos_seqs)
#     negatives = [s for i,s,c in zip(ids,seqs,clusters) if i==0 and c in pos_clusters and s not in pos_seqs]
    
#     rng.shuffle(negatives)
#     negatives = negatives[:len(sequences)]
    
#     assert len(negatives) == len(sequences)
    
#     return negatives

# Datasets

In [6]:
datasets = {}

### Mitochondrial targeting

In [7]:
df = pd.read_csv('./data/mitochondria_targeting.csv')
df = df[['Sequence','Mitochondrial Targeting Signal']].drop_duplicates(keep='first')
df = df.loc[df['Sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    sequence=df['Sequence'],
    x=df['Sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['Sequence'].map(lambda s: [1]*len(s)),
    y=df[['Mitochondrial Targeting Signal']].astype(np.int32).values
)

datasets['mito'] = dataset

len(dataset)

5349

### Cdc28 binding

In [8]:
df = pd.read_csv('./data/cdc28_binding.csv')
df = df[['Sequence','Cdc28 Binding']].drop_duplicates(keep='first')
df = df.loc[df['Sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    sequence=df['Sequence'],
    x=df['Sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['Sequence'].map(lambda s: [1]*len(s)),
    y=df[['Cdc28 Binding']].astype(np.int32).values
)

datasets['cdc28'] = dataset

len(dataset)

5348

### Signal peptide

In [9]:
with open('./data/train_set.fasta','r') as f:
    fasta = [l.strip() for l in f]
    df = pd.Series(fasta[::3]).str.extract(
        '>(?P<uniprot>[^\|]+)\|(?P<kingdom>[^|]+)\|(?P<type>[^|]+)\|(?P<partition>[^|]+)'
    )
    df['sequence'] = fasta[1::3]
    df['annotation'] = fasta[2::3]
df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    sequence=df['sequence'],
    x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['sequence'].map(lambda s: [1]*len(s)),
    y=(df['type']!='NO_SP').values[:,None].astype(int)
)

datasets['signalp'] = dataset

len(dataset)

20290

### HLA binding

In [10]:
pos_seqs = !cat ./data/mlci2012/binding_HLA-A0201.txt
neg_seqs = !cat ./data/mlci2012/nonbinding_HLA-A0201.txt
df = pd.DataFrame({
    'sequence': pos_seqs + neg_seqs,
    'hla_binding': [1]*len(pos_seqs) + [0]*len(neg_seqs)
})
df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    sequence=df['sequence'],
    x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['sequence'].map(lambda s: [1]*len(s)),
    y=(df['hla_binding']).values[:,None].astype(int)
)

datasets['hla_a0201'] = dataset

len(dataset)

11747

In [11]:
# pos_seqs = !cat ./data/mlci2012/binding_HLA-B0702.txt
# neg_seqs = !cat ./data/mlci2012/nonbinding_HLA-B0702.txt
# df = pd.DataFrame({
#     'sequence': pos_seqs + neg_seqs,
#     'hla_binding': [1]*len(pos_seqs) + [0]*len(neg_seqs)
# })
# df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
# df = df.sample(frac=1.,random_state=0)

# dataset = NamedTensorDataset(
#     sequence=df['sequence'],
#     x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
#     x_mask=df['sequence'].map(lambda s: [1]*len(s)),
#     y=(df['hla_binding']).values[:,None].astype(int)
# )

# datasets['hla_b0702'] = dataset

# len(dataset)

In [12]:
# pos_seqs = !cat ./data/mlci2012/binding_H2-Kb.txt
# neg_seqs = !cat ./data/mlci2012/nonbinding_H2-Kb.txt
# df = pd.DataFrame({
#     'sequence': pos_seqs + neg_seqs,
#     'hla_binding': [1]*len(pos_seqs) + [0]*len(neg_seqs)
# })
# df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
# df = df.sample(frac=1.,random_state=0)
# df = df.loc[df['sequence'].map(len)<=500]

# dataset = NamedTensorDataset(
#     sequence=df['sequence'],
#     x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
#     x_mask=df['sequence'].map(lambda s: [1]*len(s)),
#     y=(df['hla_binding']).values[:,None].astype(int)
# )

# datasets['h2_kb'] = dataset

# len(dataset)

### SATPDB

In [13]:
# fns = !ls ./data/satpdb/*.fasta
# df = {}
# for fn in fns:
#     name = fn.split('/')[-1].split('.')[0]
#     with open(fn,'r') as f:
#         fasta = [l.strip() for l in f]
#     df[name] = pd.Series([1]*len(fasta[::2]),index=fasta[1::2],name=name)
#     df[name] = df[name].reset_index().drop_duplicates(subset='index',keep='first')
#     df[name] = df[name].set_index('index')[name]
# df = pd.DataFrame(df).fillna(0)
# df.index.name = 'sequence'
# df = df.reset_index()
# df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
# df = df.loc[df['sequence'].map(len)>=5]
# df = df.loc[df['sequence'].map(len)<=100]

# for name in df.columns[1:]:
#     dataset = NamedTensorDataset(
#         sequence=df['sequence'],
#         x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
#         x_mask=df['sequence'].map(lambda s: [1]*len(s)),
#         y=df[name].values[:,None].astype(int)
#     )
#     datasets[name] = dataset
#     print(name,len(dataset))

In [14]:
from torch.utils.data import Subset

npr.seed(0)

MIN_LENGTH = 5
MAX_LENGTH = 100

for d in datasets:
    idxs = [
        i 
        for i,item in enumerate(datasets[d]) 
        if (len(item['sequence'])>=MIN_LENGTH) and (len(item['sequence'])<=MAX_LENGTH)
    ]
    npr.shuffle(idxs)
    datasets[d] = Subset(datasets[d], idxs)
    print(d, len(datasets[d]))

mito 4121
cdc28 4120
signalp 20290
hla_a0201 11747


# Models

In [15]:
models = {}

### MS pretraining

In [16]:
[last_ckpt] = !ls -t1 ./lightning_logs/version_17869980/checkpoints/*.ckpt | head -n1
last_ckpt

'./lightning_logs/version_17869980/checkpoints/epoch=92-step=114948-best.ckpt'

In [17]:
model = lambda : MSModel(
    checkpoint = last_ckpt,
    model_dim = 128,
    output_dim = 1,
    fixed_weights = True,
    naive = False,
    lr = 5e-4,
    output_weights = [(None,1),],
    max_length=100
)
models['ms_pretrained_frozen'] = model

In [18]:
model = lambda : MSModel(
    checkpoint = last_ckpt,
    model_dim = 128,
    output_dim = 1,
    fixed_weights = False,
    naive = False,
    lr = 5e-4,
    output_weights = [(None,1),],
    max_length=100
)
models['ms_pretrained_finetune'] = model

### Linear baseline

In [19]:
model = lambda : LinearModel(
    output_dim = 1,
    model_dim = 128,
    num_residues = len(C.alphabet),
    lr = 5e-4,
    output_weights = [(None,1),]
)
models['linear'] = model

### CNN baseline

In [20]:
# https://arxiv.org/pdf/2011.03443.pdf
# differences: 128 vs 1024, attention vs max pool
model = lambda : CNNModel(
    output_dim = 1,
    model_dim = 128,
    model_depth = 3,
    kernel_size = 5,
    num_residues = len(C.alphabet),
    dropout = 0.,
    lr = 5e-4,
    output_weights = [(None,1),]
)
models['cnn'] = model

### Random

In [21]:
model = lambda : MSModel(
    checkpoint = last_ckpt,
    model_dim = 128,
    output_dim = 1,
    fixed_weights = False,
    naive = True,
    lr = 5e-4,
    output_weights = [(None,1),],
    max_length=100
)
models['random_frozen'] = model

In [22]:
model = lambda : MSModel(
    checkpoint = last_ckpt,
    model_dim = 128,
    output_dim = 1,
    fixed_weights = True,
    naive = True,
    lr = 5e-4,
    output_weights = [(None,1),],
    max_length=100
)
models['random_finetune'] = model

### Large language model

In [23]:
model = lambda : CARPModel(
    output_dim = 1,
    fixed_weights = True,
    max_length = 100,
    lr = 5e-4,
    output_weights = [(None,1),]
)
models['carp_pretrained_frozen'] = model

In [24]:
model = lambda : CARPModel(
    output_dim = 1,
    fixed_weights = False,
    max_length = 100,
    lr = 5e-4,
    output_weights = [(None,1),]
)
models['carp_pretrained_finetune'] = model

# need to do a proper hparam search for carp

In [25]:
# for MODEL in models.keys():
#     for DATASET in datasets.keys():
#         name = MODEL+'_'+DATASET
#         !rm -rf ./lightning_logs/$name
# !rm -rf ./lightning_logs/version_$SLURM_JOBID

In [26]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from src.torch_helpers import zero_padding_collate
from src.cdhit import cdhit_split

seed_everything(0, workers=True)

aucs = {}

for MODEL in models.keys():
    for DATASET in datasets.keys():
        name = MODEL+'_'+DATASET
        !rm -rf ./lightning_logs/$name
!rm -rf ./lightning_logs/version_$SLURM_JOBID

for DATASET in datasets.keys():
    dataset = datasets[DATASET]
    
    sequences = [item['sequence'] for item in dataset]
    train_val_seqs, test_seqs, train_val_dataset, test_dataset = cdhit_split(
        sequences,
        dataset,
        split=2./3,
        threshold=0.5,
        word_length=3
    )
    
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=len(test_dataset),
        collate_fn=zero_padding_collate,
        num_workers=1,
        shuffle=False,
        drop_last=False
    )
    
    dm = PeptideDataModule(
        train_val_dataset,
        batch_size=256,
        val_batch_size=-1,
        train_val_split=0.5,
        cdhit_threshold=0.5,
        cdhit_word_length=3,
        num_workers=4
    )
    dm.setup()

    for MODEL in models.keys():
        name = MODEL+'_'+DATASET
        
        torch.manual_seed(0)
        
        model = models[MODEL]()
        
        model.output_weights = [(DATASET,1)]
        
        trainer = Trainer(
            gpus=1,
            precision=32,
            max_epochs=1000,
            min_epochs=30,
            callbacks=[
                NoValProgressBar(),
                EarlyStopping(
                    monitor=f'val_auc_{DATASET}',
                    mode='max',
                    patience=10
                ),
                ModelCheckpoint(
                    monitor=f'val_auc_{DATASET}', 
                    mode='max',
                    save_top_k=1,
                    filename='{epoch}-{step}-best'
                )
            ]
        )

        trainer.fit(model, dm)
        
        [best_ckpt] = !ls -t1 ./lightning_logs/version_$SLURM_JOBID/checkpoints/*.ckpt | head -n1
        checkpoint = torch.load(best_ckpt)
        model.load_state_dict(checkpoint['state_dict'])
        
        metrics = trainer.test(model, test_dataloader)
        
        aucs[(DATASET,MODEL)] = metrics[0][f'test_auc_{DATASET}']
        
        print(DATASET, MODEL)

        !mv ./lightning_logs/version_$SLURM_JOBID ./lightning_logs/latest
        !mv ./lightning_logs/latest ./lightning_logs/$name

Global seed set to 0
Multiprocessing is handled by SLURM.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-6f07f7d1-b74b-ade4-fb35-ff23d46bc4d9]

  | Name       | Type           | Params
----------------------------------------------
0 | encoder    | ByteNet        | 603 K 
1 | classifier | ESMAttention1d | 16.8 K
----------------------------------------------
16.8 K    Trainable params
603 K     Non-trainable params
620 K     Total params
2.481     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


/home/gridsan/mmurphy/.conda/envs/MSPretraining/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1938: PossibleUserWarning: The number of training samples (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 55: 100%|██████████| 6/6 [00:51<00:00,  8.53s/it, loss=0.0515, v_num=1.79e+7]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-6f07f7d1-b74b-ade4-fb35-ff23d46bc4d9]
/home/gridsan/mmurphy/.conda/envs/MSPretraining/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_auc_mito         0.8738208413124084
        test_loss           0.11530359834432602
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
mito ms_pretrained_frozen


...

Epoch 26: 100%|██████████| 15/15 [00:48<00:00,  3.25s/it, loss=0.137, v_num=1.79e+7]

Trainer was signaled to stop but required minimum epochs (30) or minimum steps (None) has not been met. Training will continue...


Epoch 41: 100%|██████████| 15/15 [01:14<00:00,  4.99s/it, loss=0.116, v_num=1.79e+7]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-6f07f7d1-b74b-ade4-fb35-ff23d46bc4d9]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_auc_hla_a0201       0.8103634119033813
        test_loss            1.025950312614441
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
hla_a0201 carp_pretrained_finetune


In [27]:
df = pd.DataFrame(aucs,index=[0])
df.columns.names = ['dataset','model']
df = df.T
df = df.pivot_table(index='dataset',columns='model')[0]
# df['ms_naive'] = [aucs[('cdc28','ms_naive')],aucs[('mito','ms_naive')],aucs[('signalp','ms_naive')]]
df.round(4).T

dataset,cdc28,hla_a0201,mito,signalp
model,,,,
carp_pretrained_finetune,0.8040,0.8104,0.8440,0.9940
carp_pretrained_frozen,0.7922,0.8031,0.8498,0.9915
cnn,0.6873,0.8306,0.7851,0.9260
linear,0.6379,0.7320,0.8317,0.8398
ms_pretrained_finetune,0.7582,0.8110,0.8643,0.9889
ms_pretrained_frozen,0.8024,0.7842,0.8738,0.9693
random_finetune,0.7531,0.7519,0.8543,0.9458
random_frozen,0.7658,0.8102,0.8534,0.9859


In [28]:
# from sklearn.metrics import confusion_matrix

# model = model.cpu()
# model.eval()

# ys = []
# y_preds = []

# for batch in dm.val_dataloader():
#     y_pred = model.predict_step(batch, 0).detach().cpu().numpy()
#     y = batch['y'].cpu().numpy()
#     ys.append(y)
#     y_preds.append(y_pred)
# y = np.concatenate(ys)
# y_pred = np.concatenate(y_preds)

# for k in range(y.shape[1]):
#     plt.figure(figsize=(4,4))
#     sns.heatmap(
#         confusion_matrix(y[:,k], y_pred[:,k]>0.5),
#         annot=True, fmt='d', cmap='Blues'
#     )